In [1]:
from keras.models import Sequential
from tensorflow.keras import layers
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.models import load_model
import nso_ds_classes.nso_tif_kernel as nso_tif_kernel
import nso_ds_classes.nso_ds_models as nso_ds_models
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
onehot_encoder = OneHotEncoder(sparse=False)
label_encoder = LabelEncoder()

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\geopandas\_compat.py:115: UserWarning: The Shapely GEOS version (3.4.3-CAPI-1.8.3 r4285) is incompatible with the GEOS version PyGEOS was compiled with (3.10.0-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [32]:
# Models.
def VGG_16_inspired_model(size_x_matrix =32 ,size_y_matrix = 32,bands = 4, no_classes =5):
    model = Sequential()
    model.add(Convolution2D(32, kernel_size=(3, 3), activation='relu', input_shape=(size_x_matrix,size_y_matrix,bands)))
    model.add(Convolution2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(Convolution2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(no_classes, activation='softmax'))
    return model

In [21]:
path_to_tif_file = "E:/data/coepelduynen/20210907_112017_SV1-04_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped.tif"

In [22]:
x_kernel_width = 32
y_kernel_height = 32

tif_kernel_generator = nso_tif_kernel.nso_tif_kernel_generator(path_to_tif_file, x_kernel_width , y_kernel_height)

In [23]:
a_generic_model = nso_ds_models.generic_model(tif_kernel_generator)

In [24]:
annotations = a_generic_model.get_annotations(path_to_tif_file.split("/")[-1])

In [25]:
annotations[["label","kernel"]]

,label,kernel
0,gras_hoog,"[[[365, 369, 368, 365, 361, 358, 362, 367, 370..."
1,bos,"[[[350, 348, 347, 346, 345, 339, 331, 324, 324..."
2,zand_hoog,"[[[595, 597, 598, 599, 594, 591, 591, 591, 590..."
3,gras_laag,"[[[373, 375, 374, 373, 375, 375, 370, 366, 368..."
4,zand_laag,"[[[584, 583, 579, 576, 580, 556, 498, 407, 370..."


In [ ]:
model = VGG_16_inspired_model()
model.compile(loss="sparse_categorical_crossentropy",
              optimizer= 'rmsprop',
              metrics=['accuracy'])

In [12]:
y = label_encoder.fit_transform(annotations['label'].values)
y = y.reshape(len(y), 1)
y = onehot_encoder.fit_transform(y)
print(y)

[[0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:354: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = check_array(X, dtype=np.int)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categori

In [34]:
y = label_encoder.fit_transform(annotations['label'].values)

In [58]:
annotations['label'].values

array(['gras_hoog', 'bos', 'zand_hoog', 'gras_laag', 'zand_laag '],
      dtype=object)

In [62]:
label_encoder.inverse_transform([1])

array(['gras_hoog'], dtype=object)

In [51]:
y.reshape(5,1)

array([[1],
       [0],
       [3],
       [2],
       [4]])

In [74]:
len(annotations["kernel"])

5

In [42]:
model.fit(np.concatenate(annotations["kernel"]).reshape(5,32,32,4).astype(int),y.reshape(5,1), epochs=32)

Epoch 1/32
5/5 [==============================] - 0s 38ms/step - loss: 6006.3770 - accuracy: 0.2000
Epoch 2/32
5/5 [==============================] - 0s 38ms/step - loss: 2777.5034 - accuracy: 0.2000
Epoch 3/32
5/5 [==============================] - 0s 37ms/step - loss: 634.9919 - accuracy: 0.2000
Epoch 4/32
5/5 [==============================] - 0s 38ms/step - loss: 149.1017 - accuracy: 0.4000
Epoch 5/32
5/5 [==============================] - 0s 39ms/step - loss: 22.3095 - accuracy: 0.4000
Epoch 6/32
5/5 [==============================] - 0s 38ms/step - loss: 30.7170 - accuracy: 0.8000
Epoch 7/32
5/5 [==============================] - 0s 39ms/step - loss: 38.0818 - accuracy: 0.4000
Epoch 8/32
5/5 [==============================] - 0s 39ms/step - loss: 18.0933 - accuracy: 0.8000
Epoch 9/32
5/5 [==============================] - 0s 38ms/step - loss: 15.1579 - accuracy: 0.6000
Epoch 10/32
5/5 [==============================] - 0s 39ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 11/3

In [70]:
results = model.predict(np.concatenate(annotations["kernel"][0]).reshape(1,32,32,4).astype(int))

In [73]:
np.argmax(model.predict(np.concatenate(annotations["kernel"][1]).reshape(1,32,32,4).astype(int)))

0

In [65]:
[label_encoder.inverse_transform([np.argmax(p)])[0] for p in results]

['gras_hoog', 'bos', 'zand_hoog', 'gras_laag', 'zand_laag ']

In [69]:
label_encoder.inverse_transform([np.argmax(results[0])])

array(['gras_hoog'], dtype=object)

In [154]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam

# Model configuration
batch_size = 50
img_width, img_height, img_num_channels = 32, 32, 3
loss_function = sparse_categorical_crossentropy
no_classes = 10
no_epochs = 100
optimizer = Adam()
validation_split = 0.2
verbosity = 1

# Load CIFAR-10 data
(input_train, target_train), (input_test, target_test) = cifar10.load_data()

# Determine shape of the data
input_shape = (img_width, img_height, img_num_channels)

# Parse numbers as floats
input_train = input_train.astype('float32')
input_test = input_test.astype('float32')

# Scale data
input_train = input_train / 255
input_test = input_test / 255

# Create the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(no_classes, activation='softmax'))

# Compile the model
model.compile(loss=loss_function,
              optimizer=optimizer,
              metrics=['accuracy'])

# Fit data to model
history = model.fit(input_train, target_train,
            batch_size=batch_size,
            epochs=no_epochs,
            verbose=verbosity,
            validation_split=validation_split)

# Generate generalization metrics
score = model.evaluate(input_test, target_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Train on 40000 samples, validate on 10000 samples
Epoch 1/100
 2450/40000 [>.............................] - ETA: 3:34 - loss: 2.0880 - accuracy: 0.2333

KeyboardInterrupt: 

In [156]:
input_train.shape

(50000, 32, 32, 3)

In [1]:
target_train.shape

NameError: name 'target_train' is not defined

# Try the package.

In [1]:
import nso_ds_classes.nso_tif_kernel as nso_tif_kernel
import nso_ds_classes.nso_ds_models as nso_ds_models
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\geopandas\_compat.py:115: UserWarning: The Shapely GEOS version (3.4.3-CAPI-1.8.3 r4285) is incompatible with the GEOS version PyGEOS was compiled with (3.10.0-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string
Using TensorFlow backend.


In [2]:
path_to_tif_file = "E:/data/coepelduynen/20210907_112017_SV1-04_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped.tif"

In [3]:
x_kernel_width = 32
y_kernel_height = 32

tif_kernel_generator = nso_tif_kernel.nso_tif_kernel_generator(path_to_tif_file, x_kernel_width , y_kernel_height)

In [4]:
model = nso_ds_models.deep_learning_model(tif_kernel_generator)

In [5]:
model.get_annotations(path_to_tif_file.split("/")[-1])

In [6]:
model.set_standard_convolutional_network()

In [7]:
model.train_model_on_sat_anno(path_to_tif_file.split("/")[-1])

Epoch 1/32
5/5 [==============================] - 0s 60ms/step - loss: 137.0657 - accuracy: 0.2000
Epoch 2/32
5/5 [==============================] - 0s 39ms/step - loss: 23561.5664 - accuracy: 0.2000
Epoch 3/32
5/5 [==============================] - 0s 38ms/step - loss: 8492.7666 - accuracy: 0.2000
Epoch 4/32
5/5 [==============================] - 0s 38ms/step - loss: 526.6615 - accuracy: 0.2000
Epoch 5/32
5/5 [==============================] - 0s 38ms/step - loss: 1783.0417 - accuracy: 0.2000
Epoch 6/32
5/5 [==============================] - 0s 39ms/step - loss: 178.4460 - accuracy: 0.2000
Epoch 7/32
5/5 [==============================] - 0s 39ms/step - loss: 139.1850 - accuracy: 0.2000
Epoch 8/32
5/5 [==============================] - 0s 38ms/step - loss: 72.2629 - accuracy: 0.2000
Epoch 9/32
5/5 [==============================] - 0s 38ms/step - loss: 45.5011 - accuracy: 0.4000
Epoch 10/32
5/5 [==============================] - 0s 39ms/step - loss: 28.3208 - accuracy: 0.4000
Epoch 11

In [8]:
model

In [9]:
x_row = 16
y_row = 4757

In [10]:
kernel = tif_kernel_generator.get_kernel_for_x_y(x_row,y_row )

In [12]:
np.concatenate(kernel).reshape(1,32,32,4).shape

(1, 32, 32, 4)

In [13]:
model.predict(kernel)

'bos'

In [19]:
str(type(model))

"<class 'nso_ds_classes.nso_ds_models.deep_learning_model'>"

In [21]:
if str(type(model)) == "<class 'nso_ds_classes.nso_ds_models.deep_learning_model'>":
    print("Hallo")

Hallo
